Created 2018-08-17
by Hirotaka Iwaki

# Problem setting
* 4 observations.
* x = 1, 2, 3, 4
* Y = 9, 13, 32, 41
$$Y = \beta_0 x^0 + \beta_1 x^1 + \beta_2 x^2 + \beta_3 x^3$$

# Create Desing Matrix
x = 1 -> [1,1,1,1]    
x = 2 -> [1,2,4,8]    
x = 3 -> [1,3,9,27]    
x = 4 -> [1,4,16,64]    
$\beta=[\beta_0,\beta_1,\beta_2,\beta_3]^T$
$$Y = X\beta + \epsilon$$

Usually we use normal equation to solve this.

$$X^T Y =X^T X\beta$$

$$(X^T X)^{-1}X^T Y =(X^T X)^{-1}X^T X\beta$$

$$\beta=(X^T X)^{-1}X^T Y$$

Now let's make design matrix in python

In [1]:
import numpy as np
x= np.array([1,2,3,4])
X = np.vstack((x**k for k in range(4)))
Xt = np.transpose(X)
print(Xt)

[[ 1  1  1  1]
 [ 1  2  4  8]
 [ 1  3  9 27]
 [ 1  4 16 64]]


# QR decomposition

QR decompostion is to solve the normal eqution without calclulating an inverse matrix

Any $m\ \times\ n$ matrix can be decomposed to a $m\ \times\ m$ orthogonal matrix and a $m\ \times\ n$ upper triangular 

$$X=QR$$

Orthogonal means, 
$$Q^T Q= Q Q^{T} = I$$

so 
$$Q^T =Q^{-1}$$


In [2]:
Q,R = np.linalg.qr(Xt)
print (Q)

[[-0.5         0.67082039  0.5         0.2236068 ]
 [-0.5         0.2236068  -0.5        -0.67082039]
 [-0.5        -0.2236068  -0.5         0.67082039]
 [-0.5        -0.67082039  0.5        -0.2236068 ]]


In [3]:
print(R)

[[ -2.          -5.         -15.         -50.        ]
 [  0.          -2.23606798 -11.18033989 -46.51021393]
 [  0.           0.           2.          15.        ]
 [  0.           0.           0.          -1.34164079]]


In [4]:
np.matmul(Q,R)

array([[ 1.,  1.,  1.,  1.],
       [ 1.,  2.,  4.,  8.],
       [ 1.,  3.,  9., 27.],
       [ 1.,  4., 16., 64.]])

In [5]:
Qt = np.transpose(Q)
np.matmul(Q, Qt)

array([[ 1.00000000e+00, -2.05343406e-17,  8.62196630e-17,
        -5.55111512e-17],
       [-2.05343406e-17,  1.00000000e+00,  0.00000000e+00,
        -1.39367340e-17],
       [ 8.62196630e-17,  0.00000000e+00,  1.00000000e+00,
         8.70292897e-17],
       [-5.55111512e-17, -1.39367340e-17,  8.70292897e-17,
         1.00000000e+00]])

In [6]:
np.matmul(Q, Qt).astype('float32')

array([[ 1.0000000e+00, -2.0534340e-17,  8.6219662e-17, -5.5511151e-17],
       [-2.0534340e-17,  1.0000000e+00,  0.0000000e+00, -1.3936734e-17],
       [ 8.6219662e-17,  0.0000000e+00,  1.0000000e+00,  8.7029287e-17],
       [-5.5511151e-17, -1.3936734e-17,  8.7029287e-17,  1.0000000e+00]],
      dtype=float32)

In [7]:
np.linalg.det(Q)

-0.9999999999999999

# Solve the problem without using inverse matrix

Normal equation
$$X^T Y =X^T X\beta$$

can be converted as follows.

$$(QR)^T Y = (QR)^T QR \beta$$

$$R^T Q^T Y = R^T Q^T Q R \beta$$

$$R^T Q^T Y = R^T R \beta$$

Now R^T is a triangular so it has $(R^T)^{-1}$

$$(R^T)^{-1} R^T Q^T Y = (R^T)^{-1} R^T R \beta$$

$$Q^T Y = R\beta$$

$R$ is an upper triangular matrix, so we can calculate beta one by one from the bottom raw.    

(e.g.https://en.wikipedia.org/wiki/QR_decomposition, http://metodososcaruis.blogspot.com/)

# How do we use it for longitudinal data?

Get the $Qi$ of which the first column of $Q$ is omitted.    
$Qi$ is not an orthogonal matrix anymore.

In [9]:
Qi = Q[:,1:]
Qit = np.transpose(Qi)
print(Qi)

[[ 0.67082039  0.5         0.2236068 ]
 [ 0.2236068  -0.5        -0.67082039]
 [-0.2236068  -0.5         0.67082039]
 [-0.67082039  0.5        -0.2236068 ]]


In [10]:
np.matmul(Qit, Qi)

array([[ 1.00000000e+00,  0.00000000e+00, -1.53429048e-17],
       [ 0.00000000e+00,  1.00000000e+00,  8.32667268e-17],
       [-1.53429048e-17,  8.32667268e-17,  1.00000000e+00]])

In [11]:
np.matmul(Qi, Qit)

array([[ 0.75, -0.25, -0.25, -0.25],
       [-0.25,  0.75, -0.25, -0.25],
       [-0.25, -0.25,  0.75, -0.25],
       [-0.25, -0.25, -0.25,  0.75]])

See what happens to a constant vector multiplied with $Qi^T$

In [15]:
constant = np.array([1,1,1,1])
constant_t = np.transpose(constant)
np.matmul(Qit, constant_t)

array([-1.11022302e-16, -1.66533454e-16,  5.55111512e-17])

**I still don't know why** but this $Qi^T$ transfers constant vactor to $O$    


Now, consider we are going to evaluate the association between a SNP and the decline in some test score $Y$ after 65 years old    
Our model is, 
$$ Y = \beta_0 + \beta_1 x_{sex} + \beta_2 x_{education} + \beta_3 x_{SNP} + \beta_4 x_{age-65} + \beta_5 x_{SNP} * x_{age-65}$$

$$ (Yi = Xi\beta) $$

We will think about $i$th subject who was observed at age 65, 66, 67 and 70    
The observed design matrix is;

In [16]:
Xi = np.array([[1, 1, 17, 2, 0, 0],
               [1, 1, 17, 2, 1, 2],
               [1, 1, 17, 2, 2, 4],
               [1, 1, 17, 2, 5, 10]])

We are interested in the $\beta$ of interaction term ($\beta_5$)    
Multiply the above equation with $Qi^T$

$$ Qi^T Yi = Qi^T Xi\beta $$

Let's call the above as "equation in the transformed space"
.    
the results of $Qi^T Xi$ is;

In [17]:
transXi=np.matmul(Qit, Xi)
np.around(transXi, 3)

array([[-0.   , -0.   , -0.   , -0.   , -3.578, -7.155],
       [-0.   , -0.   , -0.   , -0.   ,  1.   ,  2.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.447, -0.894]])

So it is apparent that in the transformed spcae, intercept, $x_{sex}$, $x_{education}$, $x_{SNP}$ are all 0s.    
Because they don't have any information, we cannot estimate $\beta_{0-3}$.    
But we can still estimate $\beta_4$ and $\beta_5$.    
In other words, all the cross-sectional features are omitted in this space and we don't need to think about it anymore.    
This is particularly helpful when we are not sure about cross-sectional fetures affecting an outcome.     
(we are free from worrring about whether baseline scores, number of kids or any other factors should be included in the model or not.)    
The down side is that the number of datapoints are also reduced by 1 (4 -> 3)

The above example is just illustrating the $i$th subject (with 4 observations).      
For whole data analysis, we will transfer each subject observations with their own $Qi^T$ and then solve using an ordinary linear mixed model algorithm (Mixed model because errors within a subject are thought to be correlated). Note that intercept term is not needed anymore in that linear mixed model. 

This is called **conditional linear mixed regression model**    


# Appendix

poly function to get $Q_i$     

(equivalent to R's poly function; https://stackoverflow.com/questions/41317127/python-equivalent-to-r-poly-function)

In [18]:
def poly(x, p):
    x = np.array(x)
    X = np.transpose(np.vstack((x**k for k in range(p+1))))
    return np.linalg.qr(X)[0][:,1:]

In [19]:
poly([1,2,3,4],3)

array([[ 0.67082039,  0.5       ,  0.2236068 ],
       [ 0.2236068 , -0.5       , -0.67082039],
       [-0.2236068 , -0.5       ,  0.67082039],
       [-0.67082039,  0.5       , -0.2236068 ]])

# Reference


The American Statistician Vol. 55, No. 1, Feb., 2001    
Conditional Linear Mixed Models    
Geert Verbeke, Bart Spiessens and Emmanuel Lesaffre    
Vol. 55, No. 1 (Feb., 2001), pp. 25-34    
https://www.jstor.org/stable/2685526


Eur J Hum Genet. 2015 Oct; 23(10): 1384–1391.    
Published online 2015 Feb 25. doi:  10.1038/ejhg.2015.1    
PMCID: PMC4592098    
PMID: 25712081    
GWAS with longitudinal phenotypes: performance of approximate procedures    
Karolina Sikorska,1,2 Nahid Mostafavi Montazeri,1,3 André Uitterlinden,2 Fernando Rivadeneira,2 Paul HC Eilers,1 and Emmanuel Lesaffre1,4,*
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4592098/

example R code from the above article.


            id	y	Time
            1	1.12	1
            1	1.14	2
            1	1.16	3
            1	1.2	4
            1	1.26	5
            2	0.95	1
            2	0.83	2
            2	0.65	3
            2	0.49	4
            2	0.34	5

    cond = function(data, vars) {
    data = data[order(data$id), ]
    ### delete missing observations
    data1 = data[!is.na(data$y), ]
    ## do the transformations
    ids = unique(data1$id)
    transdata = NULL
    for(i in ids) {
    xi = data1[data1$id == i, vars]
    xi = as.matrix(xi)
    if(nrow(xi) > 1) {
    A = cumsum(rep(1, nrow(xi)))
    A1 = poly(A, degree = length(A)-1)
    transxi = t(A1) %*% xi
    transxi = cbind(i, transxi)
    transdata = rbind(transdata, transxi)
        }
    }
    transdata = as.data.frame(transdata)
    names(transdata) = c("id", vars)
    row.names(transdata) = 1:nrow(transdata)
    return(transdata)
    }

    trdata = cond(mydata, vars = c("Time", "y"))
    #fit the reduced model and extract random slopes
    mod2 = lmer(y ˜ Time - 1 + (Time-1|id), data = trdata)